## Instalando Bibliotecas
Lembre de retirar o '#' para "descomentar" essa linha, para realizar a instalação das bibliotecas <br>
necessárias, após instalado uma vez, pode comentar novamente.

In [1]:
#pip install openmeteo-requests requests-cache retry-requests openpyxl

## Imports
Realizando os imports necessários para usar as API's

In [2]:
import json
import requests
import pandas as pd
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

## Questão 1:
Quantos feriados há no Brasil em todo o ano de 2024?

Usei a API Nager.DATE ( Public Holidays )<br>
Aqui usei a API e laço for para contabilizar a quantidade de feriados no ano de 2024

In [3]:
response = requests.get('https://date.nager.at/api/v3/publicholidays/2024/BR')
public_holidays = json.loads(response.content)
feriados_2024 = 0

for public_holiday in public_holidays:
  feriados_2024 += 1
  
print(feriados_2024)

14


Resposta: Em 2024 teremos 14 feriados

## Questão 2:
Qual mês de 2024 tem o maior número de feriados?

Nessa primeira parte eu decidi criar um DataFrame com os feriados, para que eu pudesse ir utilizando<br>
essa base de dados durante as questões.

In [4]:
df_feriados = pd.DataFrame(columns=['Holiday Name', 'Date'])
rows = []

for public_holiday in public_holidays:
  rows.append({'Holiday Name': public_holiday['name'], 'Nome Local': public_holiday['localName'],'Date': public_holiday['date'],})
  
df_feriados = pd.concat([df_feriados, pd.DataFrame(rows)], ignore_index=True)
df_feriados['Date'] = pd.to_datetime(df_feriados['Date']) + pd.DateOffset(hours=3)
df_feriados['Month'] = df_feriados['Date'].dt.month_name()
df_feriados['Day'] = df_feriados['Date'].dt.weekday
df_feriados['Weekday'] = df_feriados['Date'].dt.day_name()

Visualização do DataFrame

In [5]:
df_feriados.head(30)

,Holiday Name,Date,Nome Local,Month,Day,Weekday
0,New Year's Day,2024-01-01 03:00:00,Confraternização Universal,January,0,Monday
1,Carnival,2024-02-12 03:00:00,Carnaval,February,0,Monday
2,Carnival,2024-02-13 03:00:00,Carnaval,February,1,Tuesday
3,Good Friday,2024-03-29 03:00:00,Sexta-feira Santa,March,4,Friday
4,Easter Sunday,2024-03-31 03:00:00,Domingo de Páscoa,March,6,Sunday
5,Tiradentes,2024-04-21 03:00:00,Dia de Tiradentes,April,6,Sunday
6,Labour Day,2024-05-01 03:00:00,Dia do Trabalhador,May,2,Wednesday
7,Corpus Christi,2024-05-30 03:00:00,Corpus Christi,May,3,Thursday
8,Constitutionalist Revolution of 1932,2024-07-09 03:00:00,Revolução Constitucionalista de 1932,July,1,Tuesday
9,Independence Day,2024-09-07 03:00:00,Dia da Independência,September,5,Saturday


Aqui onde de fato eu fiz a captura dos meses que tinham mais feriados no ano de 2024.<br>
Usei dt.month para capturar apenas o mês da coluna Date, que foi passada para datetime anteriormente<br>
e como havia meses com a mesma quantidade de feriados, usei .max para pegar o valor máximo e printar<br>
todos que estavam no valor máximo

In [6]:
feriados_por_mes = df_feriados['Date'].dt.month.value_counts().sort_index()
max_feriados = feriados_por_mes.max()
meses_com_max_feriados = feriados_por_mes[feriados_por_mes == max_feriados]

print("Meses com o maior número de feriados:\n")
print(meses_com_max_feriados)

Meses com o maior número de feriados:

Date
2     2
3     2
5     2
11    2
Name: count, dtype: int64


Resposta: <br>
2 - Fevereiro ----- 2 feriados<br>
3 - Março ----- 2 feriados <br>
5 - Maio ----- 2 feriados<br>
11 - Novembro ----- 2 feriados<br>

## Questão 3: 
Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?


Aqui eu aproveitei o DataFrame e fiz um laço for para percorrer a coluna dias, e verificar<br>
quantos dias ficam entre segunda e sexta (entre 0 e 4).

In [7]:
count_dias_semana = 0

for dia in df_feriados['Day']:
    if 0 <= dia <= 4:
        count_dias_semana += 1

In [8]:
print(count_dias_semana)

9


Resposta: 9 feriados

## Questão 4:
Qual foi a temperatura média em cada mês?

Através do site: [Open Meteo](https://open-meteo.com/en/docs/historical-weather-api#latitude=-22.9064&longitude=-43.1822&start_date=2024-01-01&end_date=2024-08-01&hourly=&daily=temperature_2m_mean&timezone=America%2FSao_Paulo) <br>
Eu consegui achar como realizar as configurar para trabalhar com a API. 

In [9]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": -22.9064,
	"longitude": -43.1822,
	"start_date": "2024-01-01",
	"end_date": "2024-08-01",
	"daily": ["weather_code", "temperature_2m_mean"],
	"timezone": "America/Sao_Paulo"
}

responses = openmeteo.weather_api(url, params=params)

response = responses[0]
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()

daily_data = {"Date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = False),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = False),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_dataframe = pd.DataFrame(data = daily_data)

daily_data["weather_code"] = daily_weather_code
daily_data['Ano'] = daily_dataframe['Date'].dt.year
daily_data['Mês'] = daily_dataframe['Date'].dt.month
daily_data["Temperatura Média"] = daily_temperature_2m_mean

daily_dataframe = pd.DataFrame(data = daily_data)


daily_dataframe.head(50)

,Date,weather_code,Ano,Mês,Temperatura Média
0,2024-01-01 03:00:00,51.0,2024,1,24.854582
1,2024-01-02 03:00:00,51.0,2024,1,25.885834
2,2024-01-03 03:00:00,63.0,2024,1,26.223335
3,2024-01-04 03:00:00,61.0,2024,1,24.869164
4,2024-01-05 03:00:00,51.0,2024,1,25.183752
5,2024-01-06 03:00:00,51.0,2024,1,25.402502
6,2024-01-07 03:00:00,3.0,2024,1,25.983749
7,2024-01-08 03:00:00,1.0,2024,1,28.962919
8,2024-01-09 03:00:00,51.0,2024,1,29.317085
9,2024-01-10 03:00:00,53.0,2024,1,29.946245


Continuando a resolução, aqui eu criei um DataFrame para as médias mensais

Peguei as informações do outro DataFrame e agrupei os dados usando o mês como métrica, calculando a média de temperatura para cada mês

In [10]:
df_medias_mensais = pd.DataFrame(columns=['Mês', 'Ano', 'Temperatura Média'])

df_medias_mensais = daily_dataframe.groupby(['Ano', 'Mês'])['Temperatura Média'].mean().reset_index()

df_medias_mensais.head(10)

,Ano,Mês,Temperatura Média
0,2024,1,26.694233
1,2024,2,27.212988
2,2024,3,26.416479
3,2024,4,25.169930
4,2024,5,24.990740
5,2024,6,22.648958
6,2024,7,21.105591
7,2024,8,21.225414


Resposta: <br>

Janeiro: 26.69 ºC<br>
Fevereiro: 27.21 ºC<br>
Março: 26.41 ºC<br>
Abril: 25.16 ºC<br>
Maio: 24.99 ºC<br>
Junho: 22.64 ºC<br>
Julho: 21.10 ºC<br>
Agosto: 21,22 ºC<br>

## Questão 5:
Qual foi o tempo predominante em cada mês nesse período?

Aqui eu fiz uma função utilizando o método mode() para retornar o weather_code mais recorrente<br>
e agrupei usando Ano e Mês como filtro, aplicando a função em função do weather_code<br>
e fiz o merge do dataframe medias mensais e o novo dataframe criado

In [11]:
def weather_code_recorrente(valores):
    frequencia = valores.mode()
    return frequencia.iloc[0] if not frequencia.empty else None

weather_code_comum = daily_dataframe.groupby(['Ano', 'Mês'])['weather_code'].apply(weather_code_recorrente).reset_index()
weather_code_comum.rename(columns={'weather_code': 'Weather_Code_Mais_Recorrente'}, inplace=True)


df_resultado = pd.merge(df_medias_mensais, weather_code_comum, on=['Ano', 'Mês'])

df_resultado.head(10)

,Ano,Mês,Temperatura Média,Weather_Code_Mais_Recorrente
0,2024,1,26.694233,63.0
1,2024,2,27.212988,51.0
2,2024,3,26.416479,51.0
3,2024,4,25.169930,3.0
4,2024,5,24.990740,51.0
5,2024,6,22.648958,1.0
6,2024,7,21.105591,3.0
7,2024,8,21.225414,2.0


Aqui eu "carreguei" o JSON ao nosso código e fiz uma função, para fazer o mapping<br>
do weather_code do meu DataFrame com os valores que estão no JSON.

In [12]:
with open('descriptions.json', 'r') as file:
    weather_code_dict = json.load(file)

def pegar_Descrição_JSON(code, time_of_day='day'):
    if pd.isna(code):  # Verifica se o código é NaN
        return 'Descrição não disponível'
    code_str = str(int(code))
    if code_str in weather_code_dict:
        return weather_code_dict[code_str][time_of_day]['description']
    else:
        return 'Descrição não disponível'


df_resultado['Descrição do Tempo'] = df_resultado['Weather_Code_Mais_Recorrente'].apply(lambda x: pegar_Descrição_JSON(x, 'day'))

df_resultado.head(10)

,Ano,Mês,Temperatura Média,Weather_Code_Mais_Recorrente,Descrição do Tempo
0,2024,1,26.694233,63.0,Rain
1,2024,2,27.212988,51.0,Light Drizzle
2,2024,3,26.416479,51.0,Light Drizzle
3,2024,4,25.169930,3.0,Cloudy
4,2024,5,24.990740,51.0,Light Drizzle
5,2024,6,22.648958,1.0,Mainly Sunny
6,2024,7,21.105591,3.0,Cloudy
7,2024,8,21.225414,2.0,Partly Cloudy


Resposta: Foi tomado como base para todos os dias, a parte da manhã<br>
<br>
Janeiro - Rain<br>
Fevereiro - Light Drizzle<br>
Março - Light Drizzle<br>
Abril - Cloudy<br>
Maio - Light Drizzle<br>
Junho - Mainly Sunny<br>
Julho - Cloudy<br>
Agosto - Partly Cloudy<br>

## Questão 6:
Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?

Aqui eu fiz o merge do dataframe das temperaturas com os feriados.

In [13]:
df_feriados_temp = pd.merge(df_feriados, daily_dataframe[['Date', 'Temperatura Média', 'weather_code']], on='Date', how='left')

Adicionei as Descrições do tempo

In [14]:
df_feriados_temp['Descrição do Tempo'] = df_feriados_temp['weather_code'].apply(lambda x: pegar_Descrição_JSON(x, 'day'))

In [15]:
df_feriados_temp.tail(15)

,Holiday Name,Date,Nome Local,Month,Day,Weekday,Temperatura Média,weather_code,Descrição do Tempo
0,New Year's Day,2024-01-01 03:00:00,Confraternização Universal,January,0,Monday,24.854582,51.0,Light Drizzle
1,Carnival,2024-02-12 03:00:00,Carnaval,February,0,Monday,30.156668,1.0,Mainly Sunny
2,Carnival,2024-02-13 03:00:00,Carnaval,February,1,Tuesday,30.531670,51.0,Light Drizzle
3,Good Friday,2024-03-29 03:00:00,Sexta-feira Santa,March,4,Friday,25.044168,61.0,Light Rain
4,Easter Sunday,2024-03-31 03:00:00,Domingo de Páscoa,March,6,Sunday,24.531664,53.0,Drizzle
5,Tiradentes,2024-04-21 03:00:00,Dia de Tiradentes,April,6,Sunday,23.087919,2.0,Partly Cloudy
6,Labour Day,2024-05-01 03:00:00,Dia do Trabalhador,May,2,Wednesday,28.108751,0.0,Sunny
7,Corpus Christi,2024-05-30 03:00:00,Corpus Christi,May,3,Thursday,20.779579,51.0,Light Drizzle
8,Constitutionalist Revolution of 1932,2024-07-09 03:00:00,Revolução Constitucionalista de 1932,July,1,Tuesday,21.487915,53.0,Drizzle
9,Independence Day,2024-09-07 03:00:00,Dia da Independência,September,5,Saturday,NaN,NaN,Descrição não disponível


Aqui eu fiz um filtro eu guardei em um novo dataframe

In [16]:
feriados_filtro = df_feriados_temp[
    (df_feriados_temp['Date'] >= '2024-01-01') &
    (df_feriados_temp['Date'] <= '2024-08-01')
]

feriados_filtro.head(14)

,Holiday Name,Date,Nome Local,Month,Day,Weekday,Temperatura Média,weather_code,Descrição do Tempo
0,New Year's Day,2024-01-01 03:00:00,Confraternização Universal,January,0,Monday,24.854582,51.0,Light Drizzle
1,Carnival,2024-02-12 03:00:00,Carnaval,February,0,Monday,30.156668,1.0,Mainly Sunny
2,Carnival,2024-02-13 03:00:00,Carnaval,February,1,Tuesday,30.531670,51.0,Light Drizzle
3,Good Friday,2024-03-29 03:00:00,Sexta-feira Santa,March,4,Friday,25.044168,61.0,Light Rain
4,Easter Sunday,2024-03-31 03:00:00,Domingo de Páscoa,March,6,Sunday,24.531664,53.0,Drizzle
5,Tiradentes,2024-04-21 03:00:00,Dia de Tiradentes,April,6,Sunday,23.087919,2.0,Partly Cloudy
6,Labour Day,2024-05-01 03:00:00,Dia do Trabalhador,May,2,Wednesday,28.108751,0.0,Sunny
7,Corpus Christi,2024-05-30 03:00:00,Corpus Christi,May,3,Thursday,20.779579,51.0,Light Drizzle
8,Constitutionalist Revolution of 1932,2024-07-09 03:00:00,Revolução Constitucionalista de 1932,July,1,Tuesday,21.487915,53.0,Drizzle


Resposta: <br>
Ano Novo : 24.84 ºC e Light Drizzle<br>
Carnaval (2024-02-12) : 30.15 ºC e Mainly Sunny<br>
Carnaval (2024-02-13) : 30.53 ºC e Light Drizzle<br>
Sexta-feira Santa: 25.04 ºC e Light Rain<br>
Páscoa: 24.53 ºC e Drizzle<br>
Tiradentes: 23.08 ºC e Partly Cloudy<br>
Dia do Trabalho: 28.10 ºC e Sunny<br>
Corpus Christi: 20.77 ºC e Light Drizzle<br>
Revolução Constitucionalista de 1932: 21.48 ºC e Drizzle<br>

## Questão 7:
Considere as seguintes suposições:

- O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
- Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
- O cidadão carioca só vai à praia quando não está com frio;
- O cidadão carioca também só vai à praia em dias com sol, evitando dias totalmente nublados ou chuvosos (considere weather_code para determinar as condições climáticas).
<br>
<br>
Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?



Fiz um filtro utilizando com métrica temperatura abaixo de 20 e condição de tempo acima de 2 ( Parcialmente nublado )

In [17]:
feriados_ruins = df_feriados_temp[
    (df_feriados_temp['Temperatura Média'] < 20) | 
    (df_feriados_temp['weather_code'] > 2)
]
count_feriados_ruins = len(feriados_ruins)

print(f"Em 2024 teremos {count_feriados_ruins} feriados ruins para praia.")
feriados_ruins.head(15)

Em 2024 teremos 6 feriados ruins para praia.


,Holiday Name,Date,Nome Local,Month,Day,Weekday,Temperatura Média,weather_code,Descrição do Tempo
0,New Year's Day,2024-01-01 03:00:00,Confraternização Universal,January,0,Monday,24.854582,51.0,Light Drizzle
2,Carnival,2024-02-13 03:00:00,Carnaval,February,1,Tuesday,30.531670,51.0,Light Drizzle
3,Good Friday,2024-03-29 03:00:00,Sexta-feira Santa,March,4,Friday,25.044168,61.0,Light Rain
4,Easter Sunday,2024-03-31 03:00:00,Domingo de Páscoa,March,6,Sunday,24.531664,53.0,Drizzle
7,Corpus Christi,2024-05-30 03:00:00,Corpus Christi,May,3,Thursday,20.779579,51.0,Light Drizzle
8,Constitutionalist Revolution of 1932,2024-07-09 03:00:00,Revolução Constitucionalista de 1932,July,1,Tuesday,21.487915,53.0,Drizzle


Resposta:<br>

Eu usei como paramêtro as condições até no máximo parcialmente nublado, onde temos sol ainda<br>
assim, tiveram 6 feriados não aproveitáveis para o carioca

## Questão 8:
Qual foi o feriado "mais aproveitável" de 2024?

Aqui eu decidi ordenar por temperatura e weather_code, de forma que a ordenação por temperatura fosse decrescente e por weather_code<br>
fosse crescente. Após isso, eu filtrei o weather_code <= 2

In [18]:
feriado_mais_aproveital = df_feriados_temp.sort_values(by=['Temperatura Média', 'weather_code'], ascending=[False, True])

feriado_mais_aproveital.head(15)

feriado_aproveitavel_filtro = feriado_mais_aproveital[
    (feriado_mais_aproveital['weather_code'] <= 2)
]

linha_result = feriado_aproveitavel_filtro.head(1)
valor = linha_result['Nome Local'].values[0]
print(f"O feriado mais aproveitável foi: {valor}")

feriado_aproveitavel_filtro.head()

O feriado mais aproveitável foi: Carnaval


,Holiday Name,Date,Nome Local,Month,Day,Weekday,Temperatura Média,weather_code,Descrição do Tempo
1,Carnival,2024-02-12 03:00:00,Carnaval,February,0,Monday,30.156668,1.0,Mainly Sunny
6,Labour Day,2024-05-01 03:00:00,Dia do Trabalhador,May,2,Wednesday,28.108751,0.0,Sunny
5,Tiradentes,2024-04-21 03:00:00,Dia de Tiradentes,April,6,Sunday,23.087919,2.0,Partly Cloudy


Dessa forma, o feriado mais aproveitavel foi o carnaval no dia 12-02-2024